## Instalação das bibliotecas nescessarias

In [ ]:
!pip install -U google-generativeai

In [ ]:
!pip install gradio

## Importando as bibliotecas necessarias


In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import gradio as gr

In [ ]:
import google.generativeai as genai
from google.colab import userdata

## Dando acesso do drive ao colab

In [ ]:
drive.mount('/content/drive')

##caminhos dos dados de sintomas e dos historicos

In [ ]:
caminho_sintomas = "/content/drive/MyDrive/testeCLoud/Sintomas.csv"
caminho_historico = "/content/drive/MyDrive/testeCLoud/historico_medico_cloud.csv"

##Criação dos dataframes de sintomas e de historico

In [ ]:
df_sintomas = pd.read_csv(caminho_sintomas)
df_sintomas

In [ ]:
df_historico = pd.read_csv(caminho_historico)
df_historico


In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Função para gerar embeddings
def gerar_embeddings(texto):
    result = genai.embed_content(
        model="models/embedding-001",
        content=texto,
        timeout=120,
        task_type="retrieval_document",
        title="Embedding de uma sintoma"),
    return result['embedding']

In [ ]:
df_sintomas['Embeddings_Sintomas'] = df_sintomas['Sintoma'].apply(lambda x: gerar_embeddings(x))

In [ ]:
df_sintomas

In [ ]:
def identificar_sintomas(sintomas_usuario,nome_usuario):
  # Gerar embeddings dos sintomas do usuário
    embeddings_usuario = gerar_embeddings(sintomas_usuario)

    # Calcular similaridade (cosseno) entre os embeddings dos sintomas do usuário e os embeddings dos sintomas no DataFrame
    similaridades_doencas = cosine_similarity([embeddings_usuario], df_sintomas['Embeddings_Doencas'])[0]
    indice_doenca_mais_provavel = np.argmax(similaridades_doencas)

    doenca_mais_provavel = df_sintomas.iloc[indice_doenca_mais_provavel]

    resultado = {
        "Nome da Doença": doenca_mais_provavel["Nome da Doença"],
        "Descrição do Diagnóstico Médico Possível": doenca_mais_provavel["Descrição"],
        "Tratamentos": doenca_mais_provavel["Tratamentos"],
        "Médicos que podem tratar": doenca_mais_provavel["Médicos"],
        "Doenças Relacionadas ao Histórico": []
    }
    # Verificar histórico (se existir)
    if nome_usuario in df_historico['nome'].values:
        historico_usuario = df_historico[df_historico['nome'] == nome_usuario]['Historico_Medico'].iloc[0]
        embeddings_historico = gerar_embeddings(historico_usuario)
        similaridades_historico = cosine_similarity([embeddings_usuario], [embeddings_historico])[0][0]

        if similaridades_historico > 0.5:  # Defina um limiar de similaridade
            doencas_historico = df_historico[df_historico['Nome'] == nome_usuario]['Nome da Doença'].tolist()
            resultado["Doenças Relacionadas ao Histórico"] = doencas_historico

    return resultado

    return {
        "Nome da Doença": sintoma_mais_semelhante,
        "Descrição do Diagnóstico Médico Possível": descricao_mais_semelhante
    }


In [ ]:
def gradio_diagnosticar(sintomas, nome_usuario="Usuário não informado"):
    resultado = identificar_sintomas(sintomas)

    resposta = f"""
    **Nome da Doença:** {resultado['Nome da Doença']}
    **Descrição:** {resultado['Descrição do Diagnóstico Médico Possível']}
    **Tratamentos:** {resultado['Tratamentos']}
    **Médicos que podem tratar:** {resultado['Médicos que podem tratar']}
    """

    if resultado["Doenças Relacionadas ao Histórico"]:
        resposta += f"\n**Doenças Relacionadas ao Histórico:** {', '.join(resultado['Doenças Relacionadas ao Histórico'])}"

    return resposta

In [ ]:
sintomas_teste = "Perda de memória, confusão, alterações de humor"

In [ ]:
identificar_sintomas(sintomas_teste,"kevin")

In [ ]:
demo = gr.Interface(
    fn=gradio_diagnosticar,
    inputs=[
        gr.Textbox(label="Sintomas", placeholder="Descreva seus sintomas aqui"),
        gr.Textbox(label="Nome do Paciente", placeholder="Digite seu nome")
    ],
    outputs=gr.Textbox(label="Diagnóstico"),
    title="Sistema de Diagnóstico Médico",
    description="Digite seus sintomas e nome para receber um possível diagnóstico."
)

demo.launch()